### Подключение к БД и загрузка данных

In [ ]:
import pandas as pd

In [ ]:
# подключисмся к БД, закроем логин и пароль, в следующих коммитах сделать загрузку логина и пароля из файла git ignor

CONN = "postgresql://robot-startml-ro:***"\
    "postgres.lab.karpov.courses:***"

Таблица user_data
Cодержит информацию о всех пользователях соц.сети

age	Возраст пользователя (в профиле)
city	Город пользователя (в профиле)
country	Страна пользователя (в профиле)
exp_group	Экспериментальная группа: некоторая зашифрованная категория
gender	Пол пользователя
id	Уникальный идентификатор пользователя
os	Операционная система устройства, с которого происходит пользование соц.сетью
source	Пришел ли пользователь в приложение с органического трафика или с рекламы

Таблица post_text_df
Содержит информацию о постах и уникальный ID каждой единицы с соответствующим ей текстом и топиком

id	Уникальный идентификатор поста
text	Текстовое содержание поста
topic	Основная тематика

Таблица feed_data
Содержит историю о просмотренных постах для каждого юзера в изучаемый период.

timestamp	Время, когда был произведен просмотр
user_id	id пользователя, который совершил просмотр
post_id	id просмотренного поста
action	Тип действия: просмотр или лайк
target	1 у просмотров, если почти сразу после просмотра был совершен лайк, иначе 0. У действий like пропущенное значение.

In [5]:
### Данные по пользователям

user_info = pd.read_sql(
    """SELECT * FROM public.user_data""",
    
    con=CONN
)

user_info.head()

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


In [6]:
### Данные по постам

posts_info = pd.read_sql(
    """SELECT * FROM public.post_text_df""",
    
    con=CONN
)

posts_info.head()

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business


In [7]:
### Проверим кол-во записей в ленте
### Почти 77 миллионов записей
### Большой объем данных для обучения на ПК

count_feed_data = pd.read_sql(
    """SELECT count(*) FROM public.feed_data""",
    
    con=CONN
)

count_feed_data.head()

,count
0,76892800


In [8]:
### Попробуем загрузить 10 миллионов записей

feed_data = pd.read_sql(
    """SELECT * FROM public.feed_data LIMIT 10000000""",
    
    con=CONN
)

feed_data.head()

,timestamp,user_id,post_id,action,target
0,2021-12-18 21:09:54,16711,6076,view,0
1,2021-12-18 21:10:52,16711,6117,view,1
2,2021-12-18 21:11:25,16711,6117,like,0
3,2021-12-18 21:11:27,16711,3873,view,0
4,2021-12-18 21:11:39,16711,6313,view,0


### Работа с данными и фичи для контентной модели

In [9]:
### Как устроена колонка target?

feed_data[feed_data.action!='view']

,timestamp,user_id,post_id,action,target
2,2021-12-18 21:11:25,16711,6117,like,0
8,2021-12-18 21:18:15,16711,4444,like,0
13,2021-12-18 21:26:14,16711,5698,like,0
16,2021-12-18 21:30:35,16711,3940,like,0
19,2021-12-18 21:32:38,16711,7005,like,0
...,...,...,...,...,...
9999943,2021-11-19 21:52:50,119268,3226,like,0
9999950,2021-11-19 22:00:28,119268,886,like,0
9999956,2021-11-19 22:07:22,119268,1261,like,0
9999970,2021-11-25 08:18:58,119268,2166,like,0


In [10]:
### Почистим данные от action != view

feed_data = feed_data[feed_data.action == 'view']

feed_data.head()

,timestamp,user_id,post_id,action,target
0,2021-12-18 21:09:54,16711,6076,view,0
1,2021-12-18 21:10:52,16711,6117,view,1
3,2021-12-18 21:11:27,16711,3873,view,0
4,2021-12-18 21:11:39,16711,6313,view,0
5,2021-12-18 21:14:36,16711,3886,view,0


In [11]:
### Необходимо по user_id выделять 
### Фиксированный набор признаков
### В целом, подойдет оригинальный датасет

user_info

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads
...,...,...,...,...,...,...,...,...
163200,168548,0,36,Russia,Kaliningrad,4,Android,organic
163201,168549,0,18,Russia,Tula,2,Android,organic
163202,168550,1,41,Russia,Yekaterinburg,4,Android,organic
163203,168551,0,38,Russia,Moscow,3,iOS,organic


In [12]:
### Для post_id в тестовой модели
### сделаем эмбеддинг 
### Для текстов

posts_info

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business
...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie
7019,7316,I give this movie 2 stars purely because of it...,movie
7020,7317,I cant believe this film was allowed to be mad...,movie
7021,7318,The version I saw of this film was the Blockbu...,movie


In [13]:
### Сделаем эмбеддинги постов с помощью трансформера Bert или его модификаций

from transformers import AutoTokenizer
from transformers import BertModel
from transformers import RobertaModel
from transformers import DistilBertModel

# Функция выбирает вид трансформета и токенизирует текст
def get_model(model_name):
    assert model_name in ['bert', 'roberta', 'distilbert']

    checkpoint_names = {
        'bert': 'bert-base-cased',
        'roberta': 'roberta-base',
        'distilbert': 'distilbert-base-cased'
    }

    model_classes = {
        'bert': BertModel,
        'roberta': RobertaModel,
        'distilbert': DistilBertModel
    }

    return AutoTokenizer.from_pretrained(checkpoint_names[model_name]), model_classes[model_name].from_pretrained(checkpoint_names[model_name])

In [14]:
### Выбираем distilbert, в нашем случае мы можем создать эмбеддинги для всех текстов в постах,
### в проде, когда будут появляться новые тексты, distilbert обеспечит лучшую скорость обработки из выбранных моделей 

tokenizer, model = get_model('distilbert')

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
### Сделаем датасет для постов

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding


class PostDataset(Dataset):
    def __init__(self, texts, tokenizer):
        super().__init__()

        self.texts = tokenizer.batch_encode_plus(
            texts,
            add_special_tokens=True,
            return_token_type_ids=False,
            return_tensors='pt',
            truncation=True,
            padding=True
        )
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        return {'input_ids': self.texts['input_ids'][idx], 'attention_mask': self.texts['attention_mask'][idx]}

    def __len__(self):
        return len(self.texts['input_ids'])
    
    
dataset = PostDataset(posts_info['text'].values.tolist(), tokenizer)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

loader = DataLoader(dataset, batch_size=32, collate_fn=data_collator, pin_memory=True, shuffle=False)

b = next(iter(loader))

b

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[  101,  1993,  4190,  ...,     0,     0,     0],
        [  101, 15386,  1116,  ...,  1300,  1107,   102],
        [  101,  3141,   186,  ..., 14099,  8478,   102],
        ...,
        [  101, 16972, 20647,  ...,     0,     0,     0],
        [  101,   137,   188,  ...,     0,     0,     0],
        [  101,   144, 22731,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [16]:
import torch
from tqdm import tqdm

### Учим батчами, собираем вместе и конкатинируем

@torch.inference_mode()
def get_embeddings_labels(model, loader):
    model.eval()
    
    total_embeddings = []
    
    for batch in tqdm(loader):
        batch = {key: batch[key].to(device) for key in ['attention_mask', 'input_ids']}

        embeddings = model(**batch)['last_hidden_state'][:, 0, :]

        total_embeddings.append(embeddings.cpu())

    return torch.cat(total_embeddings, dim=0)

In [17]:
### Можно учить на cpu или gpu, воспользуемся free gpu с kaggle

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

model = model.to(device)

cpu


In [18]:
embeddings = get_embeddings_labels(model, loader).numpy()

embeddings

100%|██████████████████████████████████████████████████████████████████████████████| 220/220 [4:01:01<00:00, 65.74s/it]


array([[ 3.63150865e-01,  4.89375629e-02, -2.64081329e-01, ...,
        -1.41593173e-01,  1.59182400e-02,  9.18260776e-05],
       [ 2.36416146e-01, -1.59501165e-01, -3.27798277e-01, ...,
        -2.89935887e-01,  1.19365141e-01, -1.62390620e-03],
       [ 3.75191450e-01, -1.13943905e-01, -2.40547150e-01, ...,
        -3.38919193e-01,  5.86936623e-02, -2.12656148e-02],
       ...,
       [ 3.40382308e-01,  6.64923042e-02, -1.63184464e-01, ...,
        -8.65626037e-02,  2.03403741e-01,  3.20906527e-02],
       [ 4.32091981e-01,  1.10913664e-02, -1.17306121e-01, ...,
         7.54013434e-02,  1.02739811e-01,  1.52743217e-02],
       [ 3.04277569e-01, -7.62156844e-02, -6.77587166e-02, ...,
        -5.43485768e-02,  2.44383678e-01, -1.41482987e-02]], dtype=float32)

In [20]:
### Кластеризуем эмбеддинги методом главных компонент

from sklearn.decomposition import PCA

centered = embeddings - embeddings.mean()

pca = PCA(n_components=20)
pca_decomp = pca.fit_transform(centered)

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=15, random_state=0).fit(pca_decomp)

posts_info['TextCluster'] = kmeans.labels_

dists_columns = ['DistanceTo1thCluster',
                 'DistanceTo2thCluster',
                 'DistanceTo3thCluster',
                 'DistanceTo4thCluster',
                 'DistanceTo5thCluster',
                 'DistanceTo6thCluster',
                 'DistanceTo7thCluster',
                 'DistanceTo8thCluster',
                 'DistanceTo9thCluster',
                 'DistanceTo10thCluster',
                 'DistanceTo11thCluster',
                 'DistanceTo12thCluster',
                 'DistanceTo13thCluster',
                 'DistanceTo14thCluster',
                 'DistanceTo15thCluster']

dists_df = pd.DataFrame(
    data=kmeans.transform(pca_decomp),
    columns=dists_columns
)

dists_df.head()

,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,DistanceTo5thCluster,DistanceTo6thCluster,DistanceTo7thCluster,DistanceTo8thCluster,DistanceTo9thCluster,DistanceTo10thCluster,DistanceTo11thCluster,DistanceTo12thCluster,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster
0,3.504357,2.673792,1.968815,3.226219,3.331883,3.614178,3.362442,3.266323,3.291473,2.857075,3.351039,1.549979,2.108940,3.295222,3.317659
1,3.200139,2.343614,1.943182,3.177669,3.156385,3.321829,3.097685,3.251140,3.036699,2.651819,2.963483,1.431165,2.079598,2.767204,3.189289
2,3.228727,2.723816,2.829278,3.126302,3.283094,3.401746,3.286436,3.404835,3.107126,2.913411,2.971763,1.356457,2.183629,2.770222,3.223532
3,3.649482,3.229781,3.200557,3.369166,3.668101,2.867237,3.933513,3.590917,3.529238,3.132428,3.670930,2.124270,2.582012,3.554151,3.614207
4,2.821618,1.803410,2.672397,2.823427,2.552513,3.075004,3.012657,2.582009,2.659646,2.419969,2.581480,1.344970,1.539200,2.413295,2.607605


In [21]:
posts_info = pd.concat((posts_info, dists_df), axis=1)

posts_info.head()

,post_id,text,topic,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,DistanceTo5thCluster,DistanceTo6thCluster,DistanceTo7thCluster,DistanceTo8thCluster,DistanceTo9thCluster,DistanceTo10thCluster,DistanceTo11thCluster,DistanceTo12thCluster,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster
0,1,UK economy facing major risks\n\nThe UK manufa...,business,11,3.504357,2.673792,1.968815,3.226219,3.331883,3.614178,3.362442,3.266323,3.291473,2.857075,3.351039,1.549979,2.108940,3.295222,3.317659
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,11,3.200139,2.343614,1.943182,3.177669,3.156385,3.321829,3.097685,3.251140,3.036699,2.651819,2.963483,1.431165,2.079598,2.767204,3.189289
2,3,Asian quake hits European shares\n\nShares in ...,business,11,3.228727,2.723816,2.829278,3.126302,3.283094,3.401746,3.286436,3.404835,3.107126,2.913411,2.971763,1.356457,2.183629,2.770222,3.223532
3,4,India power shares jump on debut\n\nShares in ...,business,11,3.649482,3.229781,3.200557,3.369166,3.668101,2.867237,3.933513,3.590917,3.529238,3.132428,3.670930,2.124270,2.582012,3.554151,3.614207
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,11,2.821618,1.803410,2.672397,2.823427,2.552513,3.075004,3.012657,2.582009,2.659646,2.419969,2.581480,1.344970,1.539200,2.413295,2.607605


In [22]:
### Воспроизведем датафрейм со всеми новыми фичами, мержим ленту и признаки постов

df = pd.merge(feed_data,
              posts_info,
              on='post_id',
              how='left')

df.head()

,timestamp,user_id,post_id,action,target,text,topic,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,...,DistanceTo6thCluster,DistanceTo7thCluster,DistanceTo8thCluster,DistanceTo9thCluster,DistanceTo10thCluster,DistanceTo11thCluster,DistanceTo12thCluster,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster
0,2021-12-18 21:09:54,16711,6076,view,0,With a cast of stalwart British character acto...,movie,7,3.277681,1.925044,...,2.332079,3.058674,1.387538,2.252267,3.068911,1.854535,3.026086,2.788062,3.049608,1.681750
1,2021-12-18 21:10:52,16711,6117,view,1,DONT EVEN TRY to figure out the logic of this ...,movie,14,2.249643,2.535521,...,2.765206,3.448006,2.143547,1.788722,2.450539,2.058283,3.145939,2.758110,3.132016,1.738684
2,2021-12-18 21:11:27,16711,3873,view,0,Well done #NewZealand! People took it seriousl...,covid,9,1.455519,2.368171,...,2.871108,3.413861,2.437770,2.603940,1.375073,2.699581,2.782245,2.338907,3.194732,2.115251
3,2021-12-18 21:11:39,16711,6313,view,0,"Okay, some other people have commented that th...",movie,7,2.925310,2.286716,...,1.970040,3.514923,0.937262,1.732782,2.628743,1.835571,2.905313,2.508751,3.394126,0.959659
4,2021-12-18 21:14:36,16711,3886,view,0,“The scientific evidence is clear: to truly th...,covid,9,1.816763,2.696357,...,2.772372,3.513500,2.271914,2.655153,1.120753,2.569722,2.633831,2.186994,3.137830,2.257820


In [23]:
### Мержим ленту, посты, юзеров
### Получаем полный набор фичей для тестовой модели

df = pd.merge(df,
              user_info,
              on='user_id',
              how='left')

df.head()

,timestamp,user_id,post_id,action,target,text,topic,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,...,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster,gender,age,country,city,exp_group,os,source
0,2021-12-18 21:09:54,16711,6076,view,0,With a cast of stalwart British character acto...,movie,7,3.277681,1.925044,...,2.788062,3.049608,1.681750,0,21,Russia,Moscow,3,iOS,ads
1,2021-12-18 21:10:52,16711,6117,view,1,DONT EVEN TRY to figure out the logic of this ...,movie,14,2.249643,2.535521,...,2.758110,3.132016,1.738684,0,21,Russia,Moscow,3,iOS,ads
2,2021-12-18 21:11:27,16711,3873,view,0,Well done #NewZealand! People took it seriousl...,covid,9,1.455519,2.368171,...,2.338907,3.194732,2.115251,0,21,Russia,Moscow,3,iOS,ads
3,2021-12-18 21:11:39,16711,6313,view,0,"Okay, some other people have commented that th...",movie,7,2.925310,2.286716,...,2.508751,3.394126,0.959659,0,21,Russia,Moscow,3,iOS,ads
4,2021-12-18 21:14:36,16711,3886,view,0,“The scientific evidence is clear: to truly th...,covid,9,1.816763,2.696357,...,2.186994,3.137830,2.257820,0,21,Russia,Moscow,3,iOS,ads


In [24]:
### Выделим признаки из timestamp
### От времени просмотра может зависеть
### Склонность пользователей лайкать или игнорировать посты

df['hour'] = pd.to_datetime(df['timestamp']).apply(lambda x: x.hour)
df['month'] = pd.to_datetime(df['timestamp']).apply(lambda x: x.month)

df.head()

,timestamp,user_id,post_id,action,target,text,topic,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,...,DistanceTo15thCluster,gender,age,country,city,exp_group,os,source,hour,month
0,2021-12-18 21:09:54,16711,6076,view,0,With a cast of stalwart British character acto...,movie,7,3.277681,1.925044,...,1.681750,0,21,Russia,Moscow,3,iOS,ads,21,12
1,2021-12-18 21:10:52,16711,6117,view,1,DONT EVEN TRY to figure out the logic of this ...,movie,14,2.249643,2.535521,...,1.738684,0,21,Russia,Moscow,3,iOS,ads,21,12
2,2021-12-18 21:11:27,16711,3873,view,0,Well done #NewZealand! People took it seriousl...,covid,9,1.455519,2.368171,...,2.115251,0,21,Russia,Moscow,3,iOS,ads,21,12
3,2021-12-18 21:11:39,16711,6313,view,0,"Okay, some other people have commented that th...",movie,7,2.925310,2.286716,...,0.959659,0,21,Russia,Moscow,3,iOS,ads,21,12
4,2021-12-18 21:14:36,16711,3886,view,0,“The scientific evidence is clear: to truly th...,covid,9,1.816763,2.696357,...,2.257820,0,21,Russia,Moscow,3,iOS,ads,21,12


In [25]:
### Уберем все ненужные колонки

df = df.drop([
    'action',
    'text',
],
    axis=1)

df.head()

,timestamp,user_id,post_id,target,topic,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,...,DistanceTo15thCluster,gender,age,country,city,exp_group,os,source,hour,month
0,2021-12-18 21:09:54,16711,6076,0,movie,7,3.277681,1.925044,2.675330,3.429184,...,1.681750,0,21,Russia,Moscow,3,iOS,ads,21,12
1,2021-12-18 21:10:52,16711,6117,1,movie,14,2.249643,2.535521,3.498267,2.752383,...,1.738684,0,21,Russia,Moscow,3,iOS,ads,21,12
2,2021-12-18 21:11:27,16711,3873,0,covid,9,1.455519,2.368171,2.925892,2.250539,...,2.115251,0,21,Russia,Moscow,3,iOS,ads,21,12
3,2021-12-18 21:11:39,16711,6313,0,movie,7,2.925310,2.286716,3.083684,3.095057,...,0.959659,0,21,Russia,Moscow,3,iOS,ads,21,12
4,2021-12-18 21:14:36,16711,3886,0,covid,9,1.816763,2.696357,2.952883,1.927562,...,2.257820,0,21,Russia,Moscow,3,iOS,ads,21,12
5,2021-12-18 21:15:38,16711,1158,0,politics,2,3.769194,2.609201,0.939653,3.819902,...,3.409787,0,21,Russia,Moscow,3,iOS,ads,21,12
6,2021-12-18 21:17:07,16711,4444,1,movie,7,2.982104,1.987676,2.813493,3.146704,...,1.682163,0,21,Russia,Moscow,3,iOS,ads,21,12
7,2021-12-18 21:18:17,16711,1895,0,sport,13,3.215110,2.648191,3.222679,3.149311,...,3.041343,0,21,Russia,Moscow,3,iOS,ads,21,12
8,2021-12-18 21:20:53,16711,2382,0,covid,0,0.806259,2.630974,3.296063,1.937183,...,2.377984,0,21,Russia,Moscow,3,iOS,ads,21,12
9,2021-12-18 21:23:27,16711,6140,0,movie,7,3.072745,2.234633,2.988472,3.261872,...,1.086830,0,21,Russia,Moscow,3,iOS,ads,21,12


### Обучим модель

In [26]:
### На примере контрольной модели убедились, что катбуст справляется, не будем выделять тестовую выборку

X = df.drop(['target', 'timestamp'], axis=1)
Y = df['target']

In [27]:
X.head()

,user_id,post_id,topic,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,DistanceTo5thCluster,DistanceTo6thCluster,...,DistanceTo15thCluster,gender,age,country,city,exp_group,os,source,hour,month
0,16711,6076,movie,7,3.277681,1.925044,2.675330,3.429184,1.503079,2.332079,...,1.681750,0,21,Russia,Moscow,3,iOS,ads,21,12
1,16711,6117,movie,14,2.249643,2.535521,3.498267,2.752383,1.792013,2.765206,...,1.738684,0,21,Russia,Moscow,3,iOS,ads,21,12
2,16711,3873,covid,9,1.455519,2.368171,2.925892,2.250539,2.654991,2.871108,...,2.115251,0,21,Russia,Moscow,3,iOS,ads,21,12
3,16711,6313,movie,7,2.925310,2.286716,3.083684,3.095057,1.508226,1.970040,...,0.959659,0,21,Russia,Moscow,3,iOS,ads,21,12
4,16711,3886,covid,9,1.816763,2.696357,2.952883,1.927562,2.623424,2.772372,...,2.257820,0,21,Russia,Moscow,3,iOS,ads,21,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8937797,119268,792,entertainment,2,3.371973,1.927173,1.619165,3.463199,2.619300,3.070703,...,2.540460,1,25,Russia,Ust’-Ilimsk,3,iOS,organic,17,12
8937798,119268,4433,movie,14,2.525568,2.100286,3.071906,2.768228,1.441946,1.982262,...,0.652705,1,25,Russia,Ust’-Ilimsk,3,iOS,organic,17,12
8937799,119268,7263,movie,10,3.131705,2.398181,2.966780,3.253437,1.676883,2.429662,...,1.663948,1,25,Russia,Ust’-Ilimsk,3,iOS,organic,17,12
8937800,119268,6773,movie,10,2.995803,1.989803,2.975283,3.264445,1.223238,2.401585,...,1.458438,1,25,Russia,Ust’-Ilimsk,3,iOS,organic,17,12


In [28]:
### Обучим катбуст

from catboost import CatBoostClassifier


catboost = CatBoostClassifier(
    iterations=200,
    learning_rate=1,
    depth=2,
    random_seed=12345
)

object_cols = [
    'topic', 'TextCluster', 'gender', 'country',
    'city', 'exp_group', 'hour', 'month',
    'os', 'source'
]

catboost.fit(X, Y, object_cols)

0:	learn: 0.3621031	total: 18.6s	remaining: 1h 1m 47s
1:	learn: 0.3554475	total: 25.8s	remaining: 42m 32s
2:	learn: 0.3537940	total: 34.1s	remaining: 37m 16s
3:	learn: 0.3531821	total: 40.6s	remaining: 33m 11s
4:	learn: 0.3521884	total: 45.2s	remaining: 29m 24s
5:	learn: 0.3520138	total: 52.3s	remaining: 28m 11s
6:	learn: 0.3518745	total: 57s	remaining: 26m 10s
7:	learn: 0.3508866	total: 1m 1s	remaining: 24m 42s
8:	learn: 0.3505651	total: 1m 5s	remaining: 23m 4s
9:	learn: 0.3504612	total: 1m 8s	remaining: 21m 45s
10:	learn: 0.3495845	total: 1m 13s	remaining: 20m 57s
11:	learn: 0.3493934	total: 1m 17s	remaining: 20m 15s
12:	learn: 0.3492564	total: 1m 21s	remaining: 19m 31s
13:	learn: 0.3491350	total: 1m 25s	remaining: 18m 52s
14:	learn: 0.3490409	total: 1m 28s	remaining: 18m 14s
15:	learn: 0.3490065	total: 1m 33s	remaining: 17m 51s
16:	learn: 0.3487878	total: 1m 38s	remaining: 17m 37s
17:	learn: 0.3486596	total: 1m 42s	remaining: 17m 19s
18:	learn: 0.3479818	total: 1m 47s	remaining: 17m

153:	learn: 0.3441217	total: 14m 55s	remaining: 4m 27s
154:	learn: 0.3441169	total: 15m 1s	remaining: 4m 21s
155:	learn: 0.3441113	total: 15m 7s	remaining: 4m 15s
156:	learn: 0.3441011	total: 15m 13s	remaining: 4m 10s
157:	learn: 0.3440956	total: 15m 19s	remaining: 4m 4s
158:	learn: 0.3440914	total: 15m 26s	remaining: 3m 58s
159:	learn: 0.3440837	total: 15m 32s	remaining: 3m 53s
160:	learn: 0.3440787	total: 15m 38s	remaining: 3m 47s
161:	learn: 0.3440753	total: 15m 44s	remaining: 3m 41s
162:	learn: 0.3440696	total: 15m 50s	remaining: 3m 35s
163:	learn: 0.3440410	total: 15m 57s	remaining: 3m 30s
164:	learn: 0.3440332	total: 16m 3s	remaining: 3m 24s
165:	learn: 0.3440280	total: 16m 8s	remaining: 3m 18s
166:	learn: 0.3440226	total: 16m 14s	remaining: 3m 12s
167:	learn: 0.3440200	total: 16m 20s	remaining: 3m 6s
168:	learn: 0.3440159	total: 16m 26s	remaining: 3m
169:	learn: 0.3440085	total: 16m 32s	remaining: 2m 55s
170:	learn: 0.3439987	total: 16m 37s	remaining: 2m 49s
171:	learn: 0.343993

In [29]:
### Сохраним модель

catboost.save_model(
    'test_catboost_model_with_embedding'           
)

### Положим в базу фичи, необходимые для функционала тестовой модели

In [30]:
posts_info.head()

,post_id,text,topic,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,DistanceTo5thCluster,DistanceTo6thCluster,DistanceTo7thCluster,DistanceTo8thCluster,DistanceTo9thCluster,DistanceTo10thCluster,DistanceTo11thCluster,DistanceTo12thCluster,DistanceTo13thCluster,DistanceTo14thCluster,DistanceTo15thCluster
0,1,UK economy facing major risks\n\nThe UK manufa...,business,11,3.504357,2.673792,1.968815,3.226219,3.331883,3.614178,3.362442,3.266323,3.291473,2.857075,3.351039,1.549979,2.108940,3.295222,3.317659
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,11,3.200139,2.343614,1.943182,3.177669,3.156385,3.321829,3.097685,3.251140,3.036699,2.651819,2.963483,1.431165,2.079598,2.767204,3.189289
2,3,Asian quake hits European shares\n\nShares in ...,business,11,3.228727,2.723816,2.829278,3.126302,3.283094,3.401746,3.286436,3.404835,3.107126,2.913411,2.971763,1.356457,2.183629,2.770222,3.223532
3,4,India power shares jump on debut\n\nShares in ...,business,11,3.649482,3.229781,3.200557,3.369166,3.668101,2.867237,3.933513,3.590917,3.529238,3.132428,3.670930,2.124270,2.582012,3.554151,3.614207
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,11,2.821618,1.803410,2.672397,2.823427,2.552513,3.075004,3.012657,2.582009,2.659646,2.419969,2.581480,1.344970,1.539200,2.413295,2.607605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie,14,2.797614,2.126085,3.193622,2.973453,1.514944,2.135082,3.267360,1.462310,1.889674,2.631559,1.600707,2.795287,2.563891,2.826108,0.887912
7019,7316,I give this movie 2 stars purely because of it...,movie,14,2.490756,2.106853,3.082391,2.833008,1.603359,2.038254,3.287189,1.065005,1.687891,2.376269,1.682694,2.875927,2.322006,3.107956,0.677740
7020,7317,I cant believe this film was allowed to be mad...,movie,14,2.194229,2.267249,3.266811,2.676448,1.792793,2.401791,3.361870,1.799995,2.108002,2.378564,1.816606,3.063221,2.648453,3.011540,1.192410
7021,7318,The version I saw of this film was the Blockbu...,movie,7,3.242364,2.261005,3.378170,3.388438,1.055168,1.899099,3.346505,1.022737,1.922813,3.061349,1.561011,3.148641,2.936012,3.158903,1.401897


In [31]:
posts_info.to_sql(
   "bragin_posts_info_features_with_pca_embeddings",
    con=CONN,
    schema="public",
    if_exists='replace'
   )